# Nucleus-level LSTM Classifier


## Import necessary packages

In [ ]:
import os
from glob import glob

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, precision_recall_curve, PrecisionRecallDisplay

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence

## Set working directory and load datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir("/content/drive/My Drive/Colab Notebooks/")

In [ ]:
data_features = np.load("data/nucleus_level/nucleus_features.npy", allow_pickle=True)
data_labels = np.load("data/nucleus_level/nucleus_labels.npy", allow_pickle=True)

## Split into training and testing sets

In [ ]:
train_features, val_features, train_labels, val_labels = train_test_split(data_features, data_labels, test_size=0.2, random_state=1)

In [ ]:
def to_tensor(data_list):
    return [torch.FloatTensor(data) for data in data_list]

In [ ]:
train_features_input = pad_sequence(to_tensor(train_features), batch_first=True)
val_features_input = pad_sequence(to_tensor(val_features), batch_first=True)
train_labels_input = pad_sequence(to_tensor(train_labels), batch_first=True)
val_labels_input = pad_sequence(to_tensor(val_labels), batch_first=True)

In [ ]:
print(train_features_input.shape)
print(train_labels_input.shape)

torch.Size([44, 2195, 26])
torch.Size([44, 2195])


## Run sets as PyTorch Datasets through PyTorch DataLoaders

### Sets to PyTorch Datasets

In [ ]:
class trainData(Dataset):
    
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels
        
    def __getitem__(self, index):
        return self.features[index], self.labels[index]
    
    def __len__(self):
        return len(self.features)

In [ ]:
train_data = trainData(torch.FloatTensor(train_features_input), torch.FloatTensor(train_labels_input))

## Set basic training parameters

In [ ]:
EPOCHS = 50
BATCH_SIZE = 11
LEARNING_RATE = 1e-03

LENGTH = train_features_input.shape[0]
NUM_FEATURES = train_features_input.shape[2]
SEQUENCE = train_features_input.shape[1]

### Initialize DataLoaders

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE, shuffle=True)

## Ready GPU if available

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Define neural network

In [ ]:
class binaryClassifier(nn.Module):
    
    def __init__(self):
        super(binaryClassifier, self).__init__() # initialize parent class

        self.lstm_1 = nn.LSTM(input_size=NUM_FEATURES, hidden_size=128, num_layers=2, bidirectional=True, batch_first=True)

        self.dense_1 = nn.Linear(in_features=256, out_features=32)
        self.dense_2 = nn.Linear(in_features=32, out_features=1)

        self.relu = nn.ReLU()
        self.dropout_1 = nn.Dropout(0.4)
        self.dropout_2 = nn.Dropout(0.4)
                
    def forward(self, inputs):
        lstm_out, (h, c) = self.lstm_1(inputs)
        x, lengths = pad_packed_sequence(lstm_out, batch_first=True, total_length=2195)

        x = self.relu(x)
        x = self.dropout_1(x)
        x = self.dense_1(x)
        x = self.relu(x)
        x = self.dropout_2(x)
        x = self.dense_2(x)

        return x

## Initialize model and move to GPU if available

In [ ]:
model = binaryClassifier()
model.to(device)

binaryClassifier(
  (lstm_1): LSTM(26, 128, num_layers=2, batch_first=True, bidirectional=True)
  (dense_1): Linear(in_features=256, out_features=32, bias=True)
  (dense_2): Linear(in_features=32, out_features=1, bias=True)
  (relu): ReLU()
  (dropout_1): Dropout(p=0.4, inplace=False)
  (dropout_2): Dropout(p=0.4, inplace=False)
)

## Define loss function and optimizer

In [ ]:
loss_func = nn.BCEWithLogitsLoss(pos_weight=torch.tensor([3])).to(device)
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

## Define function to determine real sequence length and define metrics function

In [ ]:
def get_seq_len(seq):
  with torch.no_grad():
    lens = []
    for e in seq:
      e_no_0 = len(e[torch.where(~torch.all(torch.isclose(e, torch.cuda.FloatTensor([0])), axis=1))])
      lens.append(e_no_0)
    return lens

In [ ]:
def collect_metrics(preds, lengths, labels):
  act_sig = nn.Sigmoid()
  preds = np.concatenate([act_sig(preds[i, :lengths[i], :]).cpu().detach().numpy() for i in range(len(preds))]).flatten()
  preds_bin = np.where(preds > 0.5, 1, 0)
  labels = np.array(np.concatenate([labels[i, :lengths[i]].cpu().detach().numpy() for i in range(len(labels))]), dtype=int).flatten()
  return preds_bin, labels

## Train model

In [ ]:
model_num = 10

In [ ]:
model.train()
max_e_f1 = 0

for e in range(1, EPOCHS + 1):
    e_loss = e_f1 = 0
    e_preds_bin = e_labels = np.array([])
    
    for feature_batch, label_batch in train_loader:
        
        torch.autograd.set_detect_anomaly(True)
        feature_batch, label_batch = feature_batch.to(device), label_batch.to(device)
        optimizer.zero_grad()

        input_lengths = get_seq_len(feature_batch)

        input_features = pack_padded_sequence(feature_batch, input_lengths, batch_first=True, enforce_sorted=False)
        
        pred_labels = model(input_features)

        loss = loss_func(pred_labels, label_batch.unsqueeze(2))
        
        loss.backward()
        optimizer.step()
        
        e_loss += loss.item()

        preds_bin, labels = collect_metrics(pred_labels, input_lengths, label_batch)
        e_labels = np.concatenate([e_labels, labels])
        e_preds_bin = np.concatenate([e_preds_bin, preds_bin])

    print(classification_report(e_labels, e_preds_bin, digits=4))
    results = classification_report(e_labels, e_preds_bin, output_dict=True)

    e_f1 = results["1.0"]["f1-score"]

    print(f'Epoch {e+0:03}: | Loss: {e_loss/len(train_loader):.5f}')
    if e_f1 > max_e_f1:
      for file in glob(f"model_store/nucleus_level/model-{model_num}*"):
        os.remove(file)
      
      torch.save(model, f"model_store/nucleus_level/model-{model_num}_epoch-{e}_f1-{e_f1:.3f}.pt")
      max_e_f1 = e_f1
    else:
      pass

## Evaluate model

In [ ]:
model_file = "model-10_epoch-49_f1-0.603.pt"
eval_model = torch.load(f"model_store/nucleus_level/{model_file}").eval()

In [ ]:
val_data = trainData(torch.FloatTensor(val_features_input), torch.FloatTensor(val_labels_input))
val_loader = DataLoader(dataset=val_data, batch_size=11, shuffle=True)

In [ ]:
for feature_batch, label_batch in val_loader:
  feature_batch = feature_batch.to(device)
  input_lengths = get_seq_len(feature_batch)
  input_features = pack_padded_sequence(feature_batch, input_lengths, batch_first=True, enforce_sorted=False)
  
  
  y_pred = eval_model(input_features)
  y_true, y_pred_bin = collect_metrics(y_pred, input_lengths, label_batch)

In [ ]:
report = classification_report(y_true, y_pred_bin, digits=4)
print(report)
with open(f"eval/nucleus_level/{model_file}.txt", "w") as reportfile:
  reportfile.write(report)

              precision    recall  f1-score   support

           0     0.6907    0.9189    0.7886      7076
           1     0.8194    0.4722    0.5991      5517

    accuracy                         0.7232     12593
   macro avg     0.7551    0.6955    0.6939     12593
weighted avg     0.7471    0.7232    0.7056     12593

